In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import ShuffleSplit
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

/Users/michaelreinhardme.com/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/michaelreinhardme.com/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/michaelreinhardme.com/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/michaelreinhardme.com/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/michaelreinhardme.com/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96

Here is a link to the data: [Black Friday](https://www.kaggle.com/mehdidag/black-friday). Here is the original posting of the data set from [Analytics Vidhya](https://datahack.analyticsvidhya.com/contest/black-friday/#problem_statement).

In [2]:
import zipfile
zip_ref = zipfile.ZipFile("BlackFriday.csv.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [3]:
df = pd.read_csv('BlackFriday.csv', sep=',')

In [4]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [5]:
print(df.isna().sum())

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            166986
Product_Category_3            373299
Purchase                           0
dtype: int64


In [6]:
print(df.isna().sum()/len(df))

User_ID                       0.000000
Product_ID                    0.000000
Gender                        0.000000
Age                           0.000000
Occupation                    0.000000
City_Category                 0.000000
Stay_In_Current_City_Years    0.000000
Marital_Status                0.000000
Product_Category_1            0.000000
Product_Category_2            0.310627
Product_Category_3            0.694410
Purchase                      0.000000
dtype: float64


In [7]:
df['Age'] = df['Age'].replace(
    {'0-17':int(15), 
     '55+':int(65),
     '18-25':int(22),
     '26-35':int(31),
     '36-45':int(41),
     '46-50':int(48),
     '51-55':int(53)})

In [8]:
df['Stay_In_Current_City_Years'].head()

0     2
1     2
2     2
3     2
4    4+
Name: Stay_In_Current_City_Years, dtype: object

In [9]:
df['Stay_In_Current_City_Years'] = df['Stay_In_Current_City_Years'].replace(
    {'0':int(0), 
     '1':int(1), 
     '2':int(2), 
     '3':int(3), 
     '4+':int(4)})

In [10]:
df.groupby('Occupation')['Purchase'].mean()

Occupation
0     9186.946726
1     9017.703095
2     9025.938982
3     9238.077277
4     9279.026742
5     9388.848978
6     9336.378620
7     9502.175276
8     9576.508530
9     8714.335934
10    9052.836410
11    9299.467190
12    9883.052460
13    9424.449391
14    9568.536426
15    9866.239925
16    9457.133118
17    9906.378997
18    9233.671418
19    8754.249162
20    8881.099514
Name: Purchase, dtype: float64

In [11]:
df['City_Category'].head()

0    A
1    A
2    A
3    A
4    C
Name: City_Category, dtype: object

In [12]:
df.groupby('City_Category')['Purchase'].mean()

City_Category
A    8958.011014
B    9198.657848
C    9844.441855
Name: Purchase, dtype: float64

So there is a marginal difference between the cities and the amount of purchases people make. 

In [13]:
df.groupby('Gender')['Purchase'].mean()

Gender
F    8809.761349
M    9504.771713
Name: Purchase, dtype: float64

In [14]:
df.groupby('Marital_Status')['Purchase'].mean()

Marital_Status
0    9333.325467
1    9334.632681
Name: Purchase, dtype: float64

In [15]:
df.groupby(['Gender', 'Marital_Status'])['Purchase'].mean()

Gender  Marital_Status
F       0                 8753.809299
        1                 8887.751553
M       0                 9518.540223
        1                 9484.617891
Name: Purchase, dtype: float64

In [16]:
df.groupby(['Product_Category_1','Gender'])['Purchase'].mean()

Product_Category_1  Gender
1                   F         13597.502561
                    M         13609.885434
2                   F         11408.887314
                    M         11208.050485
3                   F         10261.916071
                    M         10027.457317
4                   F          2456.584267
                    M          2271.799625
5                   F          6305.995607
                    M          6211.920553
6                   F         15604.922235
                    M         15904.344381
7                   F         16416.834052
                    M         16359.265328
8                   F          7498.937415
                    M          7496.696382
9                   F         15724.314286
                    M         15499.311377
10                  F         19679.803163
                    M         19680.024397
11                  F          4669.676540
                    M          4686.133620
12                  F      

In [17]:
df.groupby(['City_Category', 'Gender'])['Purchase'].mean()

City_Category  Gender
A              F          8630.771856
               M          9061.717739
B              F          8590.518480
               M          9400.754481
C              F          9264.964642
               M         10033.197730
Name: Purchase, dtype: float64

In [18]:
def group_diff(df, group, mean, std_dev):
    sig_groups = []
    for index, group_mean in enumerate(df.groupby(group)['Purchase'].mean()):
        if abs(mean - 2*std_dev) < abs(mean - group_mean):
            sig_groups.append(index)
    return sig_groups

In [19]:
city_gender_groups = group_diff(df=df, group=['City_Category', 'Gender'], 
                                         mean=df['Purchase'].mean(), 
                                         std_dev=df['Purchase'].std())

print(city_gender_groups)

[0, 2, 5]


In [20]:
df.groupby('Product_Category_1')['Purchase'].mean()

Product_Category_1
1     13607.701495
2     11255.680752
3     10096.841705
4      2328.862886
5      6238.004045
6     15837.893573
7     16373.830153
8      7497.354850
9     15538.297030
10    19679.974364
11     4682.933556
12     1351.195613
13      722.619485
14    13145.452000
15    14776.422215
16    14764.157471
17    10156.440917
18     2975.307642
Name: Purchase, dtype: float64

In [21]:
df.groupby('Product_Category_2')['Purchase'].mean()

Product_Category_2
2.0     13621.740682
3.0     11229.532628
4.0     10218.319009
5.0      9034.054649
6.0     11500.585872
7.0      6877.234146
8.0     10278.036363
9.0      7282.593633
10.0    15656.014711
11.0     8935.682467
12.0     6968.662299
13.0     9672.264346
14.0     7106.356752
15.0    10358.723290
16.0    10298.676025
17.0     9416.534196
18.0     9370.698168
Name: Purchase, dtype: float64

In [22]:
# df['Product_Category_2'] = df['Product_Category_2'].fillna(df['Product_Category_2'].mode()[0]) 
df['Product_Category_2'] = df['Product_Category_2'].fillna(
    df['Product_Category_1']) 

In [23]:
# df['Product_Category_3'] = df['Product_Category_3'].fillna(df['Product_Category_3'].mode()[0])
df['Product_Category_3'] = df['Product_Category_3'].fillna(
    df['Product_Category_1']) 

In [24]:
df.groupby('Product_Category_2')['Purchase'].mean()

Product_Category_2
1.0     13074.182210
2.0     13593.102752
3.0     11270.842243
4.0     10172.640202
5.0      6957.587072
6.0     11672.275160
7.0     15028.773457
8.0      8931.445486
9.0      7284.591738
10.0    16264.057301
11.0     6947.495583
12.0     5594.340728
13.0     9168.505962
14.0     7216.709266
15.0    10704.883258
16.0    11126.641638
17.0     9447.163320
18.0     5982.959001
Name: Purchase, dtype: float64

In [25]:
df.groupby('Product_Category_3')['Purchase'].mean()

Product_Category_3
1.0     13308.386248
2.0     11304.414006
3.0     10204.445473
4.0      3895.756319
5.0      6878.908139
6.0     15155.529458
7.0     16373.830153
8.0      8083.884642
9.0     10604.236250
10.0    17936.241517
11.0     5270.981341
12.0     6581.682251
13.0     7581.147293
14.0    10290.305336
15.0    12775.395396
16.0    12627.110718
17.0    11725.388164
18.0     7759.513092
Name: Purchase, dtype: float64

In [26]:
df.describe()

,User_ID,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.375770e+05,537577.000000,537577.00000,537577.000000,537577.000000,537577.000000,537577.000000,537577.000000,537577.000000
mean,1.002992e+06,35.197002,8.08271,1.859458,0.408797,5.295546,9.159668,8.329471,9333.859853
std,1.714393e+03,11.262886,6.52412,1.289828,0.491612,3.750701,4.795474,4.760436,4981.022133
min,1.000001e+06,15.000000,0.00000,0.000000,0.000000,1.000000,1.000000,1.000000,185.000000
25%,1.001495e+06,31.000000,2.00000,1.000000,0.000000,1.000000,5.000000,5.000000,5866.000000
50%,1.003031e+06,31.000000,7.00000,2.000000,0.000000,5.000000,8.000000,8.000000,8062.000000
75%,1.004417e+06,41.000000,14.00000,3.000000,1.000000,8.000000,14.000000,12.000000,12073.000000
max,1.006040e+06,65.000000,20.00000,4.000000,1.000000,18.000000,18.000000,18.000000,23961.000000


In [27]:
y = df['Purchase']

In [28]:
#make a function to drop unused categories
def drop_col(df, use_product_factor=False, category=False):
    '''drop categories that are unused, making User_ID 
       and Product_ID optional and making Product_Category_ID optional'''
    
    if use_product_factor:   
        my_list = ['Purchase', 'User_ID', 'Product_ID']
    else:
        my_list = ['Purchase']
    if category:
        my_list.append(['Product_Category_1', 'Product_Category_2', 'Product_Category_3'])
    for col in my_list:
        try:
            df.drop(col, inplace=True, axis=1)
        except:
            'column has already been deleted'

In [29]:
drop_col(df, use_product_factor=False, category=False)

In [30]:


my_list = ['Occupation', 'Marital_Status', 'Gender', 'City_Category', \
           'Product_Category_1', 'Product_Category_2','Product_Category_3']

include_Product_ID = ['Product_ID', 'Occupation', 'Marital_Status', 'Gender', 'City_Category', \
                      'Product_Category_1', 'Product_Category_2','Product_Category_3']

exclude_Product_Category_2_3 = ['Occupation', 'Marital_Status', 'Gender', 'City_Category', \
                                'Product_Category_1']
df_qual = df.loc[:, my_list]

#df_qual = df.loc[:, include_Product_ID]
#df_qual = df.loc[:, exclude_Product_Category_2_3]

In [31]:
#make dummy variables of the qualitative variables

for var in df_qual.columns:
    df_qual = pd.concat(
                [df_qual.drop(var, axis=1), 
                 pd.get_dummies(df_qual[var], 
                   drop_first=True, 
                   prefix=var, 
                   prefix_sep='_')], 
                 axis=1)

In [32]:
df_qual.shape

(537577, 75)

In [33]:
df['Stay_In_Current_City_Years'] = df['Stay_In_Current_City_Years'].replace('4+', '4')
df['Stay_In_Current_City_Years'] = df['Stay_In_Current_City_Years'].astype('int')

In [34]:
# df['Product_ID'].dtype

In [35]:
df_quant = df.select_dtypes(['float', 'int'])

In [36]:
df_quant.columns

Index(['User_ID', 'Age', 'Occupation', 'Stay_In_Current_City_Years',
       'Marital_Status', 'Product_Category_1', 'Product_Category_2',
       'Product_Category_3'],
      dtype='object')

In [37]:
df = df_quant.join(df_qual)

In [38]:
def get_train_test_split(df, y, sample=False):
    if sample:
        df = df.sample(n=5000, random_state=42)
        y = y.sample(n=5000, random_state=42)
    
    X = df
    y = y
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test

In [39]:
X_train, X_test, y_train, y_test = get_train_test_split(df, y, sample=False)

In [40]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)  

In [ ]:
lr = LinearRegression(normalize=True)
lr.fit(X_train, y_train)
y_test_pred = lr.predict(X_test)

print(r2_score(y_test, y_test_pred))
print(np.sqrt(mean_squared_error(y_test, y_test_pred)))

In [ ]:
params={'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'] , \
        'alpha': [1, 5, 10, 20]}
ridge = GridSearchCV(Ridge(random_state=42), param_grid=params, cv=3, verbose=1, n_jobs=-1)
results = ridge.fit(X_train, y_train)
print(results.best_estimator_)
print(results.best_score_)
print(results.best_params_)

In [ ]:
ridge = Ridge(alpha=10, random_state=42)
ridge.fit(X_train, y_train)
y_test_pred = ridge.predict(X_test)

print(r2_score(y_test, y_test_pred))
print(np.sqrt(mean_squared_error(y_test, y_test_pred)))

So running grid search does nothing for the ridge regression. Well, not nothing. We increased our R^2 from 0.63598581 to 0.63598589. 

In [ ]:
params={'alpha': [30, 35, 40, 45, 50, 55, 60]}
lasso = GridSearchCV(Lasso(random_state=42, max_iter=10000, tol=0.001), param_grid=params, \
                     cv=3, verbose=1, n_jobs=-1)
results = lasso.fit(X_train, y_train)

print(results.best_estimator_)
print(results.best_score_)
print(results.best_params_)

In [ ]:
y_test_pred = lasso.predict(X_test)
print(r2_score(y_test, y_test_pred))
print(np.sqrt(mean_squared_error(y_test, y_test_pred)))

Ok, so that is another trivial improvement. I am going to try it one more time with some different values for the alpha.

In [ ]:
lasso = Lasso(random_state=42, alpha=10)
lasso.fit(X_train, y_train)
y_test_pred = lasso.predict(X_test)

print(r2_score(y_test, y_test_pred))
print(np.sqrt(mean_squared_error(y_test, y_test_pred)))

In [ ]:
params={'max_depth':[12],'max_leaf_nodes': [300],'max_features':[29], 'min_samples_leaf': [4],'n_estimators' :[400]}
cv = GridSearchCV(estimator=RandomForestRegressor(random_state=42), param_grid=params, verbose=1, cv=3, n_jobs=-1)

In [ ]:
t0 = time.time()

results = cv.fit(X_train,y_train)

t1 = time.time()

total = t1 - t0
print(total/60)

In [ ]:
print(results.best_estimator_)
print(results.best_score_)
print(results.best_params_)
y_test_pred = cv.predict(X_test)

print(r2_score(y_test, y_test_pred))
print(np.sqrt(mean_squared_error(y_test, y_test_pred)))


0.6396446905743249  
{'max_depth': 10, 'max_features': 60, 'max_leaf_nodes': 28, 'min_samples_leaf': 1}


0.6402419530153179  
{'max_depth': 10, 'max_features': 60, 'max_leaf_nodes': 30, 'min_samples_leaf': 1}  
0.6374292705606246  
2998.8808516148197  

0.6405342111257681  
{'max_depth': 10, 'max_features': 60, 'max_leaf_nodes': 32, 'min_samples_leaf': 1}  
0.6373198347718354  
2999.3333979793224  

0.6421043640764094  
{'max_depth': 10, 'max_features': 60, 'max_leaf_nodes': 37, 'min_samples_leaf': 1}  
0.639996982546654  
2988.243004415205  

0.6434113088026432  
{'max_depth': 10, 'max_features': 60, 'max_leaf_nodes': 43, 'min_samples_leaf': 1}  
0.6411755176459013  
2983.347717301678  

0.6552352176633882  
{'max_depth': 21, 'max_features': 69, 'max_leaf_nodes': 695, 'min_samples_leaf': 1, 'n_estimators': 400}  
0.6540011743719747  
2929.544864410299  

0.652088718311206  
{'max_depth': 21, 'max_features': 69, 'max_leaf_nodes': 695, 'min_samples_leaf': 1, 'n_estimators': 500}  
0.6507284171033074  
2943.3673443800467 

0.6506366033925587  
{'max_depth': 12, 'max_features': 29, 'max_leaf_nodes': 300, 'min_samples_leaf': 4, 'n_estimators': 400}  
0.6482557365667088  
2953.767805895894  

### With sample of 5000 from the data

0.6094743953931943  
{'max_depth': 12, 'max_features': 39, 'max_leaf_nodes': 400, 'min_samples_leaf': 4, 'n_estimators': 500}  
0.6187989290334923  
3000.563094011314  

0.6112096122872844  
{'max_depth': 12, 'max_features': 29, 'max_leaf_nodes': 300, 'min_samples_leaf': 4, 'n_estimators': 400}  
0.6188284074127202  
3000.447074601107  

